In [5]:
import pandas as pd
df = pd.read_excel('/Users/dejmen/desktop/ironhack/week2/day1/GSAF5.xls')
df.head()

,Date,Year,Type,Country,State,Location,Activity,Name,Sex,Age,...,Species,Source,pdf,href formula,href,Case Number,Case Number.1,original order,Unnamed: 21,Unnamed: 22
0,2025-06-17 00:00:00,2025.0,Unprovoked,USA,South Carolina,Hilton Head Island,Swimming,Not stated,F,12,...,Not stated,Kevin McMurray Trackingsharks.com:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2025-06-11 00:00:00,2025.0,Unprovoked,USA,Florida,Boca Grande,Snorkeling,Leah Lendel,F,9,...,Bull shark,Kevin McMurray Trackingsharks.com: James Kings...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-05-29 00:00:00,2025.0,Unprovoked,USA,North Carolina,Sunset Beach,Swimming,Sean Barton,M,26,...,Not stated,Kevin McMurray Trackingsharks.com: Clay Crewel...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-05-26 00:00:00,2025.0,Unprovoked,Vanuatu,South Santo,Espiitu Santo Island,Swimming,Tumas,M,14,...,Not stated,Kevin McMurray Trackingsharks.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-05-15 00:00:00,2025.0,Unprovoked,Australia,South Australia,Port Noarlunga,Swimming,Richard Vinall,M,66,...,Not stated,Simon DeMarchi: Todd Smith: 9 News:ABC News,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = pd.read_excel('GSAF5.xls')

In [ ]:
df.info()

In [ ]:
df["Type"].tail()

In [ ]:
df["Injury"].isnull().sum()

In [ ]:
# Challenges of Time feature: 
# 1 . alot of missing values
print(f"sum_of_missing_values: {df["Time"].isnull().sum()}")
print()
missing_percent = df["Time"].isnull().sum() / len(df) * 100
print(f"Percentage of missing values: {missing_percent:.2f}%")
print ()
print(f"Huge format variations: see the last 50 value counts") 
df["Time"].value_counts().tail(50)


sum_of_missing_values: 3526

Percentage of missing values: 50.23%

Huge format variations: see the last 50 value counts


Time
14h21                             1
15h53                             1
"Just before 11h00"               1
11h115                            1
12h39                             1
                                  1
N                                 1
Just before sundown               1
22h30                             1
11h30                             1
06h10                             1
Between 05h00 and 08h00           1
07h08                             1
17h00 or 17h40                    1
>08h00                            1
07h56                             1
15h56                             1
0830                              1
17h46                             1
21h50                             1
19h00, Dusk                       1
15h01                             1
1000                              1
10h44                             1
13h19                             1
Shortly before 12h00              1
17h34                             1
9h00                   

In [97]:
import pandas as pd
import re

def clean_time(value):
    if pd.isna(value):
        return None

    value = str(value).strip().lower()

    # Remove useless or unclear values
    if value in ["?", "am", "pm", "unknown", "not stated", "n/a", "na"]:
        return None

    # Clean formats like "after 1200hr", "11.30hr", "15.5", etc.
    match = re.search(r'(\d{1,2})[h:.]?(\d{2})?', value)

    if match:
        hour = match.group(1)
        minute = match.group(2) if match.group(2) else "00"
        return f"{hour.zfill(2)}:{minute.zfill(2)}"

    # Keep known phrases like "Morning", "Afternoon", etc.
    return value.title()
from datetime import datetime



In [ ]:
from datetime import datetime

def precise_time_to_day_part(value):
    if value is None:
        return None  # Keep missing as None

    # Known descriptive phrases to keep untouched
    descriptive_parts = [
        "Early Morning", "Morning", "Midday", "Early Afternoon",
        "Late Afternoon", "Afternoon", "Evening", "Dusk",
        "Night", "Late Night"
    ]
    
    if isinstance(value, str) and value.title() in descriptive_parts:
        return value.title()

    try:
        # Try parsing standard time like "14:30"
        time = datetime.strptime(value, "%H:%M").time()
        hour = time.hour
        minute = time.minute

        if 5 <= hour < 8:
            return "Early Morning"
        elif 8 <= hour < 12:
            return "Morning"
        elif hour == 12 and minute == 0:
            return "Midday"
        elif 12 <= hour < 15:
            return "Early Afternoon"
        elif 15 <= hour < 17:
            return "Late Afternoon"
        elif 17 <= hour < 19:
            return "Evening"
        elif 19 <= hour < 20:
            return "Dusk"
        elif 20 <= hour < 24:
            return "Night"
        else:  # 00:00 to before 5:00
            return "Late Night"
    except:
        return None  # Unrecognized values go to None


In [ ]:
df['Cleaned_Time'] = df['Time'].apply(clean_time)
df['Day_Part'] = df['Cleaned_Time'].apply(precise_time_to_day_part)
print(df['Day_Part'].value_counts())


In [ ]:
df['Cleaned_Time'].head(60)

In [ ]:
print(df['Day_Part'].value_counts(dropna=False))


In [ ]:
df["Date"].value_counts().tail(60)

In [ ]:
import pandas as pd
import re


def clean_date(date):
    date = str(date)
    
    # Remove known unwanted words
    cleaned_date = re.sub(r'\b(Reported|Early|Before|No date|No Date)\b', '', date, flags=re.IGNORECASE)
    cleaned_date = re.sub(r'[^0-9a-zA-Z\-/ :]', '', cleaned_date).strip()

    # Try known formats first
    for fmt in ("%d-%b-%Y", "%d %b-%Y", "%Y-%m-%d %H:%M:%S", "%d-%m-%Y", "%Y-%m-%d"):
        try:
            return pd.to_datetime(cleaned_date, format=fmt, errors='raise')
        except:
            continue

    # Fall back to automatic parsing (dayfirst off for ISO formats)
    return pd.to_datetime(cleaned_date, errors='coerce', dayfirst=False)
df['Cleaned_Date'] = df['Date'].apply(clean_date)


In [ ]:
df['Cleaned_Date'].tail(60)

In [ ]:
def get_season(date):
    if pd.isna(date):
        return "No Date"
    
    month = date.month
    if month in [3, 4, 5]:
        return "Spring"
    elif month in [6, 7, 8]:
        return "Summer"
    elif month in [9, 10, 11]:
        return "Autumn"
    elif month in [12, 1, 2]:
        return "Winter"
    
    return "No Date"

# Apply season mapping
df['Season'] = df['Cleaned_Date'].apply(get_season)

print(df[['Date', 'Cleaned_Date', 'Season']])

In [ ]:
df['Season'].value_counts()

In [ ]:
# print(df["Injury"].value_counts().to_string())


In [ ]:
counts = df["Injury"].value_counts()

# Filter where count is exactly 1
single_occurrences = counts[counts == 1]

# Show how many have count = 1
print(f"Number of unique injuries that appear only once: {len(single_occurrences)}")

In [ ]:
df["Injury_clean"] = (
    df["Injury"]
    .str.lower()
    .str.strip()
    .str.replace(r'[^a-z\s]', '', regex=True)  # Remove non-letter characters
)

# Quick grouping preview
print(df["Injury_clean"].value_counts())

In [ ]:
def simplify_injury(text):
    if pd.isna(text):
        return "unknown"
    text = text.lower()
    if "fatal" in text:
        return "fatal"
    elif "foot" in text:
        return "foot injury"
    elif "leg" in text:
        return "leg injury"
    elif "hand" in text:
        return "hand injury"
    elif "no injury" in text:
        return "no injury"
    else:
        return "other"

df["Injury_grouped"] = df["Injury"].apply(simplify_injury)
print(df["Injury_grouped"].value_counts())

In [ ]:
print(df["Injury_grouped"].value_counts().to_string())


In [ ]:
                                                 # HIPOLITO PART

#CLEAN THE FATAL COlUMN:


# Check the unique values in the column 'Fatal Y/N':
print(df['Fatal Y/N'].dropna().apply(lambda x: repr(x)).unique())

In [ ]:
#Change to string and eliminate spaces
df['Fatal Y/N'] = df['Fatal Y/N'].astype(str).str.strip().str.upper()
#Check for valid values
valid_values = {'Y': 'Y', 'N': 'N'}
#Put all the good values the rest will be NaN
df['Fatal Y/N'] = df['Fatal Y/N'].map(valid_values)
#Result
print(df['Fatal Y/N'].unique())

In [ ]:
# Count of values (Y, N)
print(df['Fatal Y/N'].value_counts(dropna=False))

In [ ]:
# Check the unique values in the column 'Country'
print(df['Country'].unique())

In [ ]:
#All in mayus and eliminate spaces
df['Country'] = df['Country'].str.strip().str.upper()
print(df['Country'].unique())


In [ ]:
# CLEANING DATA OF COUNTRIES

import numpy as np

country_corrections = {
    # Correccions Ortografics
    'COLUMBIA': 'COLOMBIA',
    'TRINIDAD & TOBAGO': 'TRINIDAD AND TOBAGO',
    'MALDIVE ISLANDS': 'MALDIVES',
    'UNITED ARAB EMIRATES (UAE)': 'UNITED ARAB EMIRATES',
    'ST. MARTIN': 'ST MARTIN',
    'ST. MAARTIN': 'ST MARTIN',
    'TRINIDAD': 'TRINIDAD AND TOBAGO',

    # Agrupations
    'ENGLAND': 'UK',
    'SCOTLAND': 'UK',
    'UNITED KINGDOM': 'UK',
    'BRITISH ISLES': 'UK',
    'BRITISH WEST INDIES': 'UK',
    'BRITISH VIRGIN ISLANDS': 'UK',

    # Ocean y region not usefull
    'PACIFIC OCEAN': 'OTHER',
    'ATLANTIC OCEAN': 'OTHER',
    'INDIAN OCEAN': 'OTHER',
    'SOUTH PACIFIC OCEAN': 'OTHER',
    'CARIBBEAN SEA': 'OTHER',
    'OCEAN': 'OTHER',
    'GULF OF ADEN': 'OTHER',
    'MID-PACIFC OCEAN': 'OTHER',
    'NORTH ATLANTIC OCEAN': 'OTHER',
    'RED SEA': 'OTHER',
    'RED SEA / INDIAN OCEAN': 'OTHER',
    'NORTH PACIFIC OCEAN': 'OTHER',
    'CENTRAL PACIFIC': 'OTHER',

    # Some other mistakes → agrupar
    'DIEGO GARCIA': 'OTHER',
    'JOHNSTON ISLAND': 'OTHER',
    'ADMIRALTY ISLANDS': 'OTHER',
    'MID ATLANTIC OCEAN': 'OTHER',
    'UNKNOWN': 'OTHER',
    'AFRICA': 'OTHER',
    'ASIA?': 'OTHER',
    'SUDAN?': 'SUDAN',
}

In [ ]:
# Import the country correction on or columns

def clean_column_country(df, column='Country'):
    # All mayus
    df[column] = df[column].str.strip().str.upper()
    #use the country_corrections to filter the column
    df[column] = df[column].replace(country_corrections)
    return df

df = clean_column_country(df, column='Country')

print(sorted(df['Country'].dropna().unique()))

In [ ]:
# Top 10 countries with more sharks attacts:
top_10_paises = df['Country'].value_counts().head(10)
print(top_10_paises)

In [6]:
activity_counts = df['Activity'].value_counts()

# Get top 10
top_10 = activity_counts.head(10)

# Calculate sum of all other activities
other_count = activity_counts[10:].sum()

# Create a new series with "Other activities" included
top_10_with_other = pd.concat([
    top_10, 
    pd.Series({'Other activities': other_count})
])

print("Top 10 activities plus others:")
print(top_10_with_other)

print("\nFormatted table:")
print(top_10_with_other.to_markdown())

Top 10 activities plus others:
Surfing             1133
Swimming             997
Fishing              490
Spearfishing         388
Wading               177
Bathing              164
Diving               147
Snorkeling           132
Standing             113
Scuba diving          84
Other activities    2610
dtype: int64

Formatted table:
|                  |    0 |
|:-----------------|-----:|
| Surfing          | 1133 |
| Swimming         |  997 |
| Fishing          |  490 |
| Spearfishing     |  388 |
| Wading           |  177 |
| Bathing          |  164 |
| Diving           |  147 |
| Snorkeling       |  132 |
| Standing         |  113 |
| Scuba diving     |   84 |
| Other activities | 2610 |


In [7]:
import pandas as pd

def get_activity_stats(df, activity_column='Activity', top_n=10):

    # Make copy to avoid modifying original DataFrame
    df = df.copy()
    
    # Standard cleaning
    df[activity_column] = (
        df[activity_column]
        .str.strip()  # Remove whitespace
        .str.lower()  # Convert to lowercase
    )
    
    # Common activity replacements (customize as needed)
    activity_replacements = {
        'swim': 'swimming',
        'bike': 'cycling',
        'bicycle': 'cycling',
        'football': 'soccer',
        'bball': 'basketball',
        'hoops': 'basketball',
        # Add more as needed for your dataset
    }
    
    df[activity_column] = df[activity_column].replace(activity_replacements)
    
    # Get activity counts
    activity_counts = df[activity_column].value_counts().reset_index()
    activity_counts.columns = ['Activity', 'Count']
    total = activity_counts['Count'].sum()
    
    # Separate top N and others
    top_activities = activity_counts.head(top_n)
    other_count = activity_counts['Count'][top_n:].sum()
    
    # Create "Other" row
    other_row = pd.DataFrame({
        'Activity': ['Other activities'],
        'Count': [other_count],
        'Percentage': [(other_count / total * 100).round(1)]
    })
    
    # Calculate percentages for top activities
    top_activities['Percentage'] = (top_activities['Count'] / total * 100).round(1)
    
    # Combine results
    result = pd.concat([top_activities, other_row], ignore_index=True)
    
    return result

# Usage example:
activity_stats = get_activity_stats(df, top_n=10)

print("Activity Statistics (Top 10 + Others):")
print(activity_stats.to_markdown(index=False, floatfmt=".1f"))

Activity Statistics (Top 10 + Others):
| Activity         |   Count |   Percentage |
|:-----------------|--------:|-------------:|
| surfing          |    1138 |         17.7 |
| swimming         |    1044 |         16.2 |
| fishing          |     506 |          7.9 |
| spearfishing     |     396 |          6.2 |
| wading           |     177 |          2.8 |
| bathing          |     167 |          2.6 |
| diving           |     150 |          2.3 |
| snorkeling       |     133 |          2.1 |
| standing         |     115 |          1.8 |
| scuba diving     |     104 |          1.6 |
| Other activities |    2505 |         38.9 |


/var/folders/74/2xg39v_50tv7vj24jw0r0b800000gn/T/ipykernel_28325/3083507710.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_activities['Percentage'] = (top_activities['Count'] / total * 100).round(1)
